# ETL Process

In [1]:
#Dependencies - Modules
import os
import csv

#Dependencies
import pandas as pd
import numpy as np

In [2]:
#Simple csv reading
'''
csvpath = os.path.join('.', 'Resources', 'ORG_OTC_FCT_DET_reduced.csv')
with open(csvpath, newline='') as csvfile:

    # CSV reader specifies delimiter and variable that holds contents
    csvreader = csv.reader(csvfile, delimiter=',')

    print(csvreader)

    # Read the header row first (skip this step if there is now header)
    csv_header = next(csvreader)
    print(f"CSV Header: {csv_header}")

    # Read each row of data after the header
    for row in csvreader:
        print(row)
'''

'\ncsvpath = os.path.join(\'.\', \'Resources\', \'ORG_OTC_FCT_DET_reduced.csv\')\nwith open(csvpath, newline=\'\') as csvfile:\n\n    # CSV reader specifies delimiter and variable that holds contents\n    csvreader = csv.reader(csvfile, delimiter=\',\')\n\n    print(csvreader)\n\n    # Read the header row first (skip this step if there is now header)\n    csv_header = next(csvreader)\n    print(f"CSV Header: {csv_header}")\n\n    # Read each row of data after the header\n    for row in csvreader:\n        print(row)\n'

In [3]:
#Using pandas to create df
#loading csv - the correct encoding must be used to read the CSV in pandas
file_to_load = "Resources/ORG_OTC_FCT_DET_reduced.csv"
original_organ_transp_data = pd.read_csv(file_to_load,encoding="ISO-8859-1")
#print(original_organ_transp_data.head)



In [4]:
#Renamed columns to load csv into MySQL - need to use '_' in between spaces
original_organ_transp_data.columns = original_organ_transp_data.columns.str.strip().str.lower().str.replace(' ', '_')
#original_organ_transp_data


In [5]:
#store csv to lead into mysql
#original_organ_transp_data.to_csv('renamed_organ_transp_data.csv')


# MySQL Process

In [6]:
# SQL Alchemy
from sqlalchemy import create_engine

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()



In [7]:
# Create Engine and Pass in MySQL Connection
engine = create_engine('sqlite:///db/organ_transplant.sqlite')
conn = engine.connect()


# APP.PY

In [8]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)



In [9]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/organ_transplant.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples


@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")


@app.route("/names")
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Return a list of the column names (sample names)
    return jsonify(list(df.columns)[2:])


@app.route("/metadata/<sample>")
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.organ_transplantation_type,
        Samples_Metadata.organ_procurement_organization_name,
        Samples_Metadata.organ_procurement_organization_city,
        Samples_Metadata.county,
        Samples_Metadata.state,
        Samples_Metadata.state_abbreviation,
        Samples_Metadata.data_warehouse_record_create_date_text,
        Samples_Metadata.geocoding__primary_x_coordinate,
        Samples_Metadata.geocoding__primary_y_coordinate,
    ]

    results = db.session.query(*sel).filter(Samples_Metadata.sample == sample).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["organ_transplantation_type"] = result[1]
        sample_metadata["organ_procurement_organization_name"] = result[2]
        sample_metadata["organ_procurement_organization_city"] = result[3]
        sample_metadata["county"] = result[4]
        sample_metadata["state"] = result[5]
        sample_metadata["state_abbreviation"] = result[6]
        sample_metadata["data_warehouse_record_create_date_text"] = result[6]
        sample_metadata["geocoding__primary_x_coordinate"] = result[6]
        sample_metadata["geocoding__primary_y_coordinate"] = result[6]

    print(sample_metadata)
    return jsonify(sample_metadata)

'''
@app.route("/samples/<sample>")
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]
    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    return jsonify(data)

'''
if __name__ == "__main__":
    app.run()

/anaconda3/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


AttributeError: sample_metadata